In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-text
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 588.3 MB 18 kB/s 
     |████████████████████████████████| 6.0 MB 52.0 MB/s 
     |████████████████████████████████| 1.7 MB 45.7 MB/s 
     |████████████████████████████████| 439 kB 63.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv')

df = df[df.Rating != 3]

df['is_positive'] = (df['Rating'] >= 4).astype(int)

In [ ]:
df.drop(columns=['Rating'], inplace=True)

In [ ]:
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df

,DATA_COLUMN,LABEL_COLUMN
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1
5,love monaco staff husband stayed hotel crazy w...,1
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",1
20487,great location price view hotel great quick pl...,1
20488,"ok just looks nice modern outside, desk staff ...",0
20489,hotel theft ruined vacation hotel opened sept ...,0


In [ ]:
df['LABEL_COLUMN'].value_counts()

1    15093
0     3214
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]

In [ ]:
df_negative = df[df['LABEL_COLUMN']==0]

In [ ]:
# Тестовая выборка

In [ ]:
n_test = (df_negative.shape[0] // 4) * 3 # в оригинале (df_negative.shape[0] // 4) * 3
df_negative_test = df_negative.tail(n_test)
n_test = (df_positive.shape[0] // 20) * 3 # в оригинале df_positive.shape[0] // 20) * 3
df_positive_test = df_positive.tail(n_test)

In [ ]:
df_positive_test

,DATA_COLUMN,LABEL_COLUMN
17565,wonderful hotel wonderful town unbeknownst man...,1
17566,wonderful wonderful wonderful just returned ye...,1
17567,"affinia 50 you'all, stayed affinia 50 week, li...",1
17568,just right good location walking distance bloo...,1
17569,loved place want thank outstanding service goi...,1
...,...,...
20480,great play stay stay loyal inn package deal ha...,1
20482,great choice wife chose best western quite bit...,1
20483,good bed clean convenient just night happy sta...,1
20486,"best kept secret 3rd time staying charm, not 5...",1


In [ ]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [ ]:
df_balanced_test.sample(10)

,DATA_COLUMN,LABEL_COLUMN
18764,"modern comfortable hotel, hotel recent vacatio...",1
15033,not great colleagues recommended stay intercon...,0
19623,simply fantastic location really fantastic hot...,1
10017,hotel theft deception just got stay hotel lang...,0
6072,"beautiful beach nothing, just got hotel stayed...",0
17954,"fantastico value, husband just returned 6 nigh...",1
19951,best service fortunate stay luxury hotels past...,1
20006,terrific staff gorgeous property travel extens...,1
18895,great resort booked trip march 31 april 8/08 1...,1
5477,"lots good hotels florence, don__Ç_é_ stay, par...",0


In [ ]:
df_balanced_test['LABEL_COLUMN'].value_counts()

0    2409
1    2262
Name: LABEL_COLUMN, dtype: int64

In [ ]:
# Обучающая выборка

In [ ]:
n_train = df_negative.shape[0] // 4 # в оригианле df_negative.shape[0] // 4
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 20 # в оригинале df_positive.shape[0] // 20
df_positive_train = df_positive.head(n_train)

In [ ]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [ ]:
df_balanced_train['LABEL_COLUMN'].value_counts()

0    803
1    754
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_balanced_train.sample(10)

,DATA_COLUMN,LABEL_COLUMN
527,"globus tour stayed globus tour, tram/ bus/trai...",1
1731,not stay not expected especially reading revie...,0
652,wonderful weekend fairmont copley plaza enjoye...,1
362,"loved stay, stayed univ, inn 10 days april 200...",1
3165,not place great group guys enjoy golf package ...,0
3346,overpriced moldy musty hotel excited going pue...,0
1023,tired worn stayed kensington park hotel 19/01/...,0
3155,"good crack second week plastered, start, impre...",0
3833,husband son stayed riu taino february 15 22. d...,0
3137,beautiful beach let start saying cleanest beau...,0


In [ ]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['LABEL_COLUMN'].squeeze()

In [ ]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

In [ ]:
talkheads_ggelu_bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3') # uses same thing as bert



In [ ]:
talkheads_ggelu_bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/2", trainable=False)

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = talkheads_ggelu_bert_preprocess(text_input)
outputs = talkheads_ggelu_bert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)


In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
49/49 [==============================] - 1015s 20s/step - loss: 0.6770 - accuracy: 0.5716 - precision: 0.5614 - recall: 0.5279
Epoch 2/10
49/49 [==============================] - 1000s 20s/step - loss: 0.5920 - accuracy: 0.7463 - precision: 0.7809 - recall: 0.6618
Epoch 3/10
49/49 [==============================] - 993s 20s/step - loss: 0.5356 - accuracy: 0.7906 - precision: 0.8213 - recall: 0.7255
Epoch 4/10
49/49 [==============================] - 1010s 21s/step - loss: 0.4993 - accuracy: 0.8131 - precision: 0.8284 - recall: 0.7745
Epoch 5/10
49/49 [==============================] - 1002s 20s/step - loss: 0.4654 - accuracy: 0.8304 - precision: 0.8582 - recall: 0.7785
Epoch 6/10
49/49 [==============================] - 1006s 21s/step - loss: 0.4429 - accuracy: 0.8343 - precision: 0.8563 - recall: 0.7905
Epoch 7/10
49/49 [==============================] - 1007s 21s/step - loss: 0.4230 - accuracy: 0.8414 - precision: 0.8487 - recall: 0.8183
Epoch 8/10
49/49 [=================

In [ ]:
y_predicted = model.predict(X_test)
y_predicted

146/146 [==============================] - 3055s 21s/step


array([[0.3642328 ],
       [0.28156686],
       [0.3152506 ],
       ...,
       [0.66129327],
       [0.72863597],
       [0.9551935 ]], dtype=float32)

In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [ ]:
y_test

4133     0
4134     0
4135     0
4136     0
4140     0
        ..
20480    1
20482    1
20483    1
20486    1
20487    1
Name: LABEL_COLUMN, Length: 4671, dtype: int64

In [ ]:
y_test_array = np.array(y_test)

In [ ]:
accuracy_score(y_test, y_predicted)

0.7966174266752302

In [ ]:
precision_score(y_test_array, y_predicted)


0.8438155136268344

In [ ]:
recall_score(y_test_array, y_predicted)

0.7117595048629531

In [ ]:
f1_score(y_test_array, y_predicted)

0.7721822541966427

In [ ]:
df_results_for_reviews_on_hotels = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [ ]:
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'precision'] = precision_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'recall'] = recall_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'f1_score'] =  f1_score(y_test_array, y_predicted)


In [ ]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/talkheads_ggelu_bert_trained_on_hotel_reviews_28_december'

In [ ]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/talkheads_ggelu_bert_trained_on_hotel_reviews_28_december


In [ ]:
model.save(saved_model_path, include_optimizer=True) 

In [ ]:
df_results_for_reviews_on_hotels


,accuracy,precision,recall,f1_score
reviews on hotels,0.796617,0.843816,0.71176,0.772182
